In [ ]:
import requests
import csv
from datetime import datetime, timedelta

START_TIME = str(datetime.now())[0:19]
PROVIDER = "newmotion"
CSV_PATH = 'D:\Web Scraping\webscrap\webscrap'
PROVIDER_PAGE = 'https://my.newmotion.com/api/map/v2'

cities = {
    #'London': {'lat': '51.668930', 'lng': '-0.489560', 'lat1': '51.349694', 'lng1': '0.279483' },
    #'Birmingham': {'lat': '52.637493', 'lng': '-2.239233', 'lat1': '52.325953', 'lng1': '-1.470876' },
    #'Paris': {'lat': '49.032292', 'lng': '1.968716', 'lat1': '48.693052', 'lng1': '2.735013' },
    'Marseille': {'lat': '43.470791', 'lng': '5.002147', 'lat1': '43.106928', 'lng1': '5.768444' },
    #'Berlin': {'lat': '52.665795', 'lng': '13.049817', 'lat1': '52.358010', 'lng1': '13.803033' },
    #'Hamburg': {'lat': '53.731440', 'lng': '9.543763', 'lat1': '53.391317', 'lng1': '10.361664' },
    #'Munich': {'lat': '48.351601', 'lng': '11.125933', 'lat1': '47.966949', 'lng1': '11.955719' },
    #'Rome': {'lat': '42.127030', 'lng': '12.131926', 'lat1': '41.7005513', 'lng1': '12.95083' },
    #'Vienna': {'lat': '48.409400', 'lng': '15.988518', 'lat1': '48.026487', 'lng1': '16.810612' },
    #'Madrid': {'lat': '40.661428', 'lng': '-4.085660', 'lat1': '40.223188', 'lng1': '-3.263954' },
}

for city, v in cities.items():
    try:
        min_lat = v['lat1']
        max_lat = v['lat']
        min_lng = v['lng']
        max_lng = v['lng1']

        url = PROVIDER_PAGE + '/markers/%s/%s/%s/%s' % (min_lng, max_lng, min_lat, max_lat)
        print("%-15s" % (city + ':'), url)
        fmt_tm = datetime.now().strftime("%Y%m%d_%H%M")
        csv_fnm = '%s_%s_%s.csv' % (PROVIDER.lower().replace(' ', '_'), city.lower().replace(' ', '_'), fmt_tm)


        r = requests.get(url)
        jsonResponse = r.json()
    except:
        continue
placeIds = []
for item in jsonResponse:
    try:
        placeIds.append(item['locationUid'])
    except:
        continue
print('\n\n### start_time:', START_TIME)
stations = []
for place in placeIds:
    detailUrl = PROVIDER_PAGE + '/locations/%s' % (place)
    detailResponse = requests.get(PROVIDER_PAGE + '/locations/%s' % (place)).json()
    station_details = {"provider":None,"city":None,"id":None,"device_type":None, "name": None, "latitude": None, "longitude":None,"address":None}
    station_details['provider'] = PROVIDER
    station_details['city'] = city
    station_details['id'] = detailResponse['uid']
    print('###station_id:', station_details['id'])
    station_details['device_type'] = detailResponse['evses'][0]['connectors'][0]['connectorType']
    station_details['name'] = detailResponse['operatorName']
    station_details['latitude'] = detailResponse['coordinates']['latitude']
    station_details['longitude'] = detailResponse['coordinates']['longitude']
    station_details['address'] = detailResponse['address']
    stations.append(station_details)

scrape_time = str(datetime.now())[0:19]
print('\n\nTotal number of charging stations:', len(stations))
print('\n\n### scrape_time:', scrape_time)

def csv_writer(outfile='newmotion.csv'): # change path
    targetfile = open(outfile, mode='w', encoding='utf-8', newline='\n')
    writer = csv.writer(targetfile, quoting=csv.QUOTE_MINIMAL)
    writer.writerow(station_details)
    return writer, targetfile

fmt_tm = datetime.now().strftime("%Y%m%d_%H%M")
csv_fnm = '%s_%s_%s.csv' % (PROVIDER.lower().replace(' ', '_'), city.lower().replace(' ', '_'), fmt_tm)

writer, targetfile = csv_writer(csv_fnm)
for st in stations:
    row_data = list(st.values())
    writer.writerow(row_data)
targetfile.close()
print('\n### CSV file closed:', csv_fnm)